In [6]:
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.metrics import roc_curve
from sklearn.metrics import auc

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score


In [7]:
df = pd.read_csv('../../data/03-week/WA_Fn-UseC_-Telco-Customer-Churn.csv')

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'] = df['TotalCharges'].fillna(0)

df.columns = df.columns.str.lower().str.replace(' ', '_')

string_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in string_columns:
    df[col] = df[col].str.lower().str.replace(' ', '_')

df.churn = (df.churn == 'yes').astype(int)

In [8]:
df_train_full, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train_full, test_size=0.33, random_state=11)

y_train = df_train.churn.values
y_val = df_val.churn.values

del df_train['churn']
del df_val['churn']

In [4]:
categorical = ['gender', 'seniorcitizen', 'partner', 'dependents',
               'phoneservice', 'multiplelines', 'internetservice',
               'onlinesecurity', 'onlinebackup', 'deviceprotection',
               'techsupport', 'streamingtv', 'streamingmovies',
               'contract', 'paperlessbilling', 'paymentmethod']
numerical = ['tenure', 'monthlycharges', 'totalcharges']

In [10]:
def train(df, y, C=1.0):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    dv.fit(cat)

    X = dv.transform(cat)

    model = LogisticRegression(solver='liblinear', C=C)
    model.fit(X, y)

    return dv, model


def predict(df, dv, model):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    X = dv.transform(cat)

    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

In [11]:

nfolds = 5
C = 1.0
kfold = KFold(n_splits=nfolds, shuffle=True, random_state=1)
aucs = []

for train_idx, val_idx in kfold.split(df_train_full):
        df_train = df_train_full.iloc[train_idx]
        df_val = df_train_full.iloc[val_idx]

        y_train = df_train.churn.values
        y_val = df_val.churn.values

        dv, model = train(df_train, y_train, C=C)
        y_pred = predict(df_val, dv, model)
        
        auc = roc_auc_score(y_val, y_pred)
        aucs.append(auc)

In [12]:
aucs

[0.8423240260300963,
 0.8453247086478611,
 0.8337066024483243,
 0.8323627454115241,
 0.8521736060995889]

In [14]:
y_train = df_train_full.churn.values
y_test = df_test.churn.values

dv, model = train(df_train_full, y_train, C=1)
y_pred = predict(df_test, dv, model)

auc = roc_auc_score(y_test, y_pred)
print(auc)

0.8579400803839363


### Save the model 

In [15]:
import pickle

In [17]:
output_file = f'model_C={C}.bin'

In [18]:
f_out = open(output_file,'wb')
pickle.dump((dv,model),f_out)
f_out.close()

In [19]:
with open(output_file,'wb') as f_out:
    pickle.dump((dv,model),f_out)

### Load the model

In [1]:
import pickle

In [2]:
model_path = 'model_C=1.0.bin'

In [3]:
with open(model_path,'rb') as file:
    (dv,model)= pickle.load(file)

In [4]:
model

LogisticRegression(C=1, solver='liblinear')

In [11]:
df_train_full.loc[0].to_json()

'{"customerid":"7590-vhveg","gender":"female","seniorcitizen":0,"partner":"yes","dependents":"no","tenure":1,"phoneservice":"no","multiplelines":"no_phone_service","internetservice":"dsl","onlinesecurity":"no","onlinebackup":"yes","deviceprotection":"no","techsupport":"no","streamingtv":"no","streamingmovies":"no","contract":"month-to-month","paperlessbilling":"yes","paymentmethod":"electronic_check","monthlycharges":29.85,"totalcharges":29.85,"churn":0}'

In [1]:
import requests

In [5]:
url = 'http://localhost:9696/predict'

In [6]:
customer = {
  "gender": "female",
  "seniorcitizen": 0,
  "partner": "yes",
  "dependents": "no",
  "tenure": 1,
  "phoneservice": "no",
  "multiplelines": "no_phone_service",
  "internetservice": "dsl",
  "onlinesecurity": "no",
  "onlinebackup": "yes",
  "deviceprotection": "no",
  "techsupport": "no",
  "streamingtv": "no",
  "streamingmovies": "no",
  "contract": "month-to-month",
  "paperlessbilling": "yes",
  "paymentmethod": "electronic_check",
  "monthlycharges": 29.85,
  "totalcharges": 29.85
}

In [10]:
responses = requests.post(url,json=customer).json()

In [12]:
responses

{'churn': False, 'churn_probability': 0.6433014487923182}

In [11]:
customer = "7590-vhveg"
if responses['churn']:
    print(f'send an email to the customer ID {customer}')